### Using Jupyter Notebook on Windows to send and remotely execute a shell script with Open SSH
This technique can be helpful in a workflow where you prepare a shell script locally on Windows and then need to execute it on one or more remote servers and have scripts output documented. Consider an example of an offloaded Oracle database upgrade where you take backup on a source system, transfer it to a target system, then restore it on target system and then upgrade on target. This scenario consists of multiple steps where each step can be scripted as an independent shell script. Each such script will have high probability of producing errors due to unforseen environmental surcumstances, making Jupiter Notebook a great tool for iterative trial and errors approach.

Starting with build 1809 Windows 10 ships with OpenSSH, with client ssh normally installed under C:\Windows\System32\OpenSSH.

The technique shown in this notebook uses OpenSSH. 
ssh is assumed to be in system PATH.

Also it is assumed that local public key had already been added to remote server authorized_hosts file.
Scroll down on how to do it.

Steps:

In [60]:
# "!" is a Notebook "magic" which runs arbitrary command in a local shell (CMD on Windows). 
# Example:
!dir aaa.sh

 Volume in drive C is Local Disk
 Volume Serial Number is 501D-27E7

 Directory of C:\Users\X271176

12/24/2019  09:04 AM                52 aaa.sh
               1 File(s)             52 bytes
               0 Dir(s)  164,894,162,944 bytes free


In [61]:
# lets create following shell script outside of Notebook.
# Script is supposed to print a parameter value and a hostname value
# We will then send this script to a remote host for execution
!type aaa.sh


echo "parameter : $1"
echo "hostname  : `hostname`"


In [62]:
# Execute local script on a remote host by redirecting standard input to remote host bash
# Note bash "-s" flag which allows you to pass parameters ("Hello" in this case)
!type "aaa.sh" | ssh -q oracle@oraracnp03 /usr/bin/bash -s Hello

parameter : Hello
hostname  : oraracnp03


In [66]:
# lets simplify above construct by creating SSH_TARGET_EXEC alias
# The alias usage will have form of "SSH_TARGET <local_script> <parameter1> <parameter2> ..."
# The "%s" placeholder is where "local_script" name will be substituted
%alias SSH_TARGET_EXEC type %s | ssh -q oracle@oraracnp03 /usr/bin/bash -s

In [69]:
# now use the alias to remotely execute aaa.sh on the remote host and pass it parameter
%SSH_TARGET_EXEC aaa.sh 'Hello there ...'

parameter : Hello there ...
hostname  : oraracnp03


#### This concludes presentation of the Notebook remote script execution technique
**Summary of Notebook Benefits for DBA workflow templating :**
- Useful in "Semi-Automation" scenarious where there is high probability of runtime failures due to unforseable environmnet impacts, making "full" automation impractical. For example, where Oracle pre-upgrade script dynamically generates manual fix recommendations different from one environment to another due to different installed options. Or when each step output has to be analyzed by a human before running next step.
- Ability to iteratively run remote scripts. You do not have to run complete workflow from start to finish as it is done with full authomation. This makes Notebook a "Semi-Automation" approach allowing you to interatively run try/error/fix/re-try cycle until you find a good solution and then apply this as a template to other targets where it may still run into other errors - and then letting you to again fix those errors.
- Allows you to create a template runbook which can be reused multiple times for different targets
- Editable and flexible, allowing you to fix steps if they break. 
- Self-documenting execution results



#### Addendum: Public/Private keys

Above demonstration assumed that local public key was added to remote hosts authorized_hosts file.
This makes ssh to not prompt for a password.

Bellow is how one can generate key pair on Windows 10 OpenSSH and then add local public key to a remote host

1. **check if user keys already exist.**
If user keys already exist then skip next step of generating the keys.
Otherwise existing keys will be overwritten and new public key will have to be resent to all hosts where it was previously sent
```
dir C:\Users\<username>\.ssh

     Volume in drive C is Local Disk
     Volume Serial Number is 501D-27E7

     Directory of C:\Users\<username>\.ssh

    12/23/2019  03:29 PM    <DIR>          .
    12/23/2019  03:29 PM    <DIR>          ..
    12/23/2019  03:29 PM             1,675 id_rsa
    12/23/2019  03:29 PM               408 id_rsa.pub
    12/23/2019  03:49 PM             1,417 known_hosts
                   3 File(s)          3,500 bytes
                   2 Dir(s)  164,874,145,792 bytes free
```

2. **If no existing keys - generate ssh keys**
```
ssh-keygen
	Generating public/private rsa key pair.
	Enter file in which to save the key (C:\Users\X271176/.ssh/id_rsa):
	Enter passphrase (empty for no passphrase):
	Enter same passphrase again:
	Your identification has been saved in C:\Users\X271176/.ssh/id_rsa.
	Your public key has been saved in C:\Users\X271176/.ssh/id_rsa.pub.
```

3. **send PUBLIC key to remote server and add it to .ssh/authorized_keys**
```
cd C:\Users\<username>\.ssh
type "id_rsa.pub" | ssh -q remote-user@remote-host-1 "cat >> .ssh/authorized_keys"
type "id_rsa.pub" | ssh -q remote-user@remote-host-2 "cat >> .ssh/authorized_keys"
...
```

4. **verify passwordless ssh works by sending "hostname" command**
```
ssh remote-user@remote-host-1 hostname
    remote-host-1
ssh remote-user@remote-host-2 hostname
    remote-host-2
```